## Model 2: Machine learning algorithm

In [ ]:
# import libraries: standard env, +pysal, + jupyter, + matplotlib
#### import libraries ####
import os
import pandas as pd
import geopandas as gpd
#### eda
import matplotlib.pyplot as plt
import seaborn as sns
from pysal.explore import esda
from pysal.lib import weights

os.getcwd()

## Clean & Combine Data

In [ ]:
# Combine processed FS shapefile w/ sars data

## Random Forest Classification

In [ ]:
#### RANDOM FOREST CLASSIFICATION ####

# A. SPLIT TRAIN/TEST SETS
# Random 70% split in train and test sets (time series is accounted for by lags)

In [ ]:
# B. OPTIMISE MODEL HYPERPARAMETERS
# Hyperparameter grid search (112)


In [ ]:
# C. RESULTS
# Fit optimised model
# Evaluate model
# Plot Confusion matrix
# Plot Variable importance
# Make prediction
# Calculate accuracy score